In [54]:
import pandas as pd
import glob

class DataLoader:
    def __init__(self, root_path: str, paths: list[str]) -> None:
        self.root: str = root_path
        self.paths: list[str] = []
        for path in paths:
            self.paths.append(self.root + path)
    
    def load_csvs(self) -> dict[str, pd.DataFrame]:
        files: dict[str, pd.DataFrame] = {}
        for path in self.paths:
            csvs: list[str] = glob.glob(path + "*.csv")
            for csv in csvs:
                print("Reading " + csv + "...", end = "")
                files[csv.split('/')[-1]] = pd.read_csv(csv)
                print(" done.")
        
        return files

In [81]:
class DataTransformer:
    def __init__(self, dataframes, label):
        self.dfs = dataframes
        self.rankings = self.transform_rankings(self.dfs)
        self.train = self.rankings[self.rankings["Season"] != 2022]
        self.test = self.rankings[self.rankings["Season"] == 2022]
        self.train = self.add_labels(self.train, label)
        self.test = self.process_test(self.test)
    
    def transform_rankings(self, data):
        ordinals = data['MMasseyOrdinals.csv']
        seasons = np.unique(ordinals["Season"])
        systems = np.unique(ordinals["SystemName"])
        final_ordinals = pd.DataFrame()
        all_finals = []
        for season in seasons:
            season_frame = ordinals.loc[ordinals["Season"] == season]
            for system in systems:
                season_system_frame = season_frame.loc[season_frame["SystemName"] == system]
                if not (season_system_frame.empty):
                    maximum_day = max(season_system_frame["RankingDayNum"])
                    season_system_finals = season_system_frame.loc[season_system_frame["RankingDayNum"] == maximum_day]
                    all_finals.append(season_system_finals)
        final_ordinals = pd.concat(all_finals, axis = 0)
        system_dfs = []
        for system in systems:
            system_dfs.append(final_ordinals.loc[final_ordinals["SystemName"] == system].drop(["SystemName", "RankingDayNum"], axis=1).rename(columns={"OrdinalRank": system}))
        joint_ordinals = system_dfs[0]
        for df in system_dfs[1:]:
            joint_ordinals = joint_ordinals.merge(df, how="outer", on=["Season", "TeamID"])
        seeds = data["MNCAATourneySeeds.csv"]
        confs = data["MTeamConferences.csv"]
        with_seeds = joint_ordinals.merge(seeds, how="inner", on=["Season", "TeamID"])
        with_conf = with_seeds.merge(confs, on=["Season", "TeamID"])
        data = with_conf
        data["Seed"] = data["Seed"].map(lambda x: int(x) if len(x) == 2 else (int(x[1:3]) if len(x) == 4 else int(x[1:])))

        return data
        
    def add_labels(self, data, label):
        results = self.dfs["MNCAATourneyCompactResults.csv"].drop(["DayNum", "WScore", "LScore", "WLoc", "NumOT"], axis = 1)
        X = []
        y = []
        for i in range(len(results)):
            result = results.iloc[i]
            team1 = min(result["WTeamID"], result["LTeamID"])
            team2 = max(result["WTeamID"], result["LTeamID"])
            season = result["Season"]
            season_data = data.loc[data["Season"] == season]
            x1 = season_data.loc[(season_data["TeamID"] == team1)]
            x2 = season_data.loc[(season_data["TeamID"] == team2)]

            # We can make our data robust to which team is considered team1 in the input
            # by making a second input with the teams swapped
            if (x1.shape[0] == 1) and (x2.shape[0] == 1):
                x = x1.merge(x2, on=["Season"], suffixes=("_team1", "_team2"))
                xr = x2.merge(x1, on=["Season"], suffixes=("_team1", "_team2"))
                X.append(x)
                X.append(xr)
                if (team1 == result["WTeamID"]):
                    y.append(1)
                    y.append(0)
                else:
                    y.append(0)
                    y.append(1)

        y = pd.Series(y, name=label, dtype=int)
        X = pd.concat(X, axis=0)
        X = X.reset_index()
        X[label] = y

        # Some validation data tinkering suggested the best way to handle missing rankings
        # is to give teams the ~lowest possible rankings. For years where an entire column is
        # missing we set it constant, for systems that don't rank all teams, we treat unranked teams
        # as bad which seems reasonable.
        X = X.fillna(350)

        return X
        
    def process_test(self, data):
        X = []
        for i in range(len(data)):
            for j in range(len(data)):
                if (i != j):
                    x1 = data.iloc[i].to_frame().T
                    x2 = data.iloc[j].to_frame().T
                    x = pd.merge(x1, x2, on=["Season"], suffixes=("_team1", "_team2"))
                    X.append(x)
        X = pd.concat(X, axis=0)
        X = X.reset_index()
        X = X.fillna(350)
        
        return X
    
    def get_train(self):
        return self.train
    
    def get_test(self):
        return self.test

In [77]:
data_path = "../data/"
stage_one_path = "MDataFiles_Stage1/"
stage_two_path = "MDataFiles_Stage2/"

data_loader = DataLoader(data_path, [stage_two_path])

files = data_loader.load_csvs()

Reading ../data/MDataFiles_Stage2/MNCAATourneyDetailedResults.csv... done.
Reading ../data/MDataFiles_Stage2/MNCAATourneyCompactResults.csv... done.
Reading ../data/MDataFiles_Stage2/MSeasons.csv... done.
Reading ../data/MDataFiles_Stage2/MMasseyOrdinals_thruDay128.csv... done.
Reading ../data/MDataFiles_Stage2/MRegularSeasonDetailedResults.csv... done.
Reading ../data/MDataFiles_Stage2/MNCAATourneySlots.csv... done.
Reading ../data/MDataFiles_Stage2/MGameCities.csv... done.
Reading ../data/MDataFiles_Stage2/MConferenceTourneyGames.csv... done.
Reading ../data/MDataFiles_Stage2/Cities.csv... done.
Reading ../data/MDataFiles_Stage2/MRegularSeasonCompactResults.csv... done.
Reading ../data/MDataFiles_Stage2/MNCAATourneySeedRoundSlots.csv... done.
Reading ../data/MDataFiles_Stage2/MTeamConferences.csv... done.
Reading ../data/MDataFiles_Stage2/MSampleSubmissionStage2.csv... done.
Reading ../data/MDataFiles_Stage2/MTeamCoaches.csv... done.
Reading ../data/MDataFiles_Stage2/MMasseyOrdinals.

In [82]:
transformer = DataTransformer(files, label="label")

In [83]:
train = transformer.get_train()
test = transformer.get_test()

In [84]:
train

,index,Season,TeamID_team1,7OT_team1,ACU_team1,ADE_team1,AP_team1,ARG_team1,ATP_team1,AUS_team1,...,WMR_team2,WMV_team2,WOB_team2,WOL_team2,WTE_team2,YAG_team2,ZAM_team2,Seed_team2,ConfAbbrev_team2,label
0,0,2003,1411,350.0,350.0,350.0,350.0,230.0,350.0,350.0,...,350.0,350.0,212.0,220.0,289.0,350.0,350.0,16,big_south,0
1,0,2003,1421,350.0,350.0,350.0,350.0,255.0,350.0,350.0,...,350.0,350.0,231.0,246.0,275.0,350.0,350.0,16,swac,1
2,0,2003,1112,350.0,350.0,350.0,2.0,2.0,350.0,350.0,...,350.0,350.0,164.0,177.0,178.0,350.0,350.0,16,aec,1
3,0,2003,1436,350.0,350.0,350.0,350.0,151.0,350.0,350.0,...,350.0,350.0,2.0,2.0,1.0,350.0,350.0,1,pac_ten,0
4,0,2003,1113,350.0,350.0,350.0,350.0,33.0,350.0,350.0,...,350.0,350.0,18.0,19.0,31.0,350.0,350.0,7,cusa,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2357,0,2021,1222,4.0,350.0,350.0,6.0,4.0,350.0,350.0,...,350.0,2.0,2.0,350.0,350.0,350.0,350.0,1,big_twelve,0
2358,0,2021,1211,1.0,350.0,350.0,1.0,1.0,350.0,350.0,...,350.0,43.0,41.0,350.0,350.0,350.0,350.0,11,pac_twelve,1
2359,0,2021,1417,48.0,350.0,350.0,350.0,32.0,350.0,350.0,...,350.0,1.0,1.0,350.0,350.0,350.0,350.0,1,wcc,0
2360,0,2021,1124,19.0,350.0,350.0,3.0,2.0,350.0,350.0,...,350.0,1.0,1.0,350.0,350.0,350.0,350.0,1,wcc,1


In [85]:
test

,index,Season,TeamID_team1,7OT_team1,ACU_team1,ADE_team1,AP_team1,ARG_team1,ATP_team1,AUS_team1,...,WLS_team2,WMR_team2,WMV_team2,WOB_team2,WOL_team2,WTE_team2,YAG_team2,ZAM_team2,Seed_team2,ConfAbbrev_team2
0,0,2022,1103,122.0,350,350,350.0,131.0,156.0,350,...,350,350,24.0,27.0,32.0,350,350,350,6,sec
1,0,2022,1103,122.0,350,350,350.0,131.0,156.0,350,...,350,350,2.0,1.0,1.0,350,350,350,1,pac_twelve
2,0,2022,1103,122.0,350,350,350.0,131.0,156.0,350,...,350,350,15.0,20.0,22.0,350,350,350,4,sec
3,0,2022,1103,122.0,350,350,350.0,131.0,156.0,350,...,350,350,5.0,4.0,3.0,350,350,350,2,sec
4,0,2022,1103,122.0,350,350,350.0,131.0,156.0,350,...,350,350,3.0,2.0,2.0,350,350,350,1,big_twelve
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4551,0,2022,1463,87.0,350,350,350.0,150.0,159.0,350,...,350,350,11.0,9.0,8.0,350,350,350,2,big_east
4552,0,2022,1463,87.0,350,350,350.0,150.0,159.0,350,...,350,350,54.0,77.0,74.0,350,350,350,11,acc
4553,0,2022,1463,87.0,350,350,350.0,150.0,159.0,350,...,350,350,16.0,7.0,10.0,350,350,350,3,big_ten
4554,0,2022,1463,87.0,350,350,350.0,150.0,159.0,350,...,350,350,204.0,212.0,215.0,350,350,350,16,horizon


In [86]:
from autogluon.tabular import TabularPredictor

In [87]:
predictor = TabularPredictor(label="label", eval_metric="log_loss").fit(train.drop(["TeamID_team1", "TeamID_team2"],axis=1))

No path specified. Models will be saved in: "AutogluonModels/ag-20220317_071532/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20220317_071532/"
AutoGluon Version:  0.4.0
Python Version:     3.9.10
Operating System:   Darwin
Train Data Rows:    2362
Train Data Columns: 380
Label Column: label
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    6525.13 MB
	Train Data (Original)  Memory Usage: 7.44 MB (0.1% of available memory)
	Inferring data type of each feat

In [90]:
y_pred = predictor.predict(test.drop(["TeamID_team1", "TeamID_team2"],axis=1))
probs = predictor.predict_proba(test.drop(["TeamID_team1", "TeamID_team2"],axis=1))

In [91]:
test["pred"] = y_pred
test["proba"] = probs[1]
names = files["MTeams.csv"]
test_with_names = test.merge(names[["TeamID", "TeamName"]], left_on="TeamID_team1", right_on="TeamID", suffixes=("", "_team1"))
test_with_names = test_with_names.merge(names[["TeamID", "TeamName"]], left_on="TeamID_team2", right_on="TeamID", suffixes=("", "_team2"))

In [92]:
def break_even_moneyline(probability):
    if probability > 0.5:
        x = -(100 / (1-probability))+100
    else:
        x = 100/probability - 100
    return x

In [100]:
def predict_probs_and_moneylines(data):
    predictions = pd.DataFrame(columns=["Team1", "Seed1", "Team2", "Seed2", "Win%1", "Win%2", "ML1", "ML2"])
    pred_array = []
    for i in range(len(data)):
        game = data[["TeamName", "TeamName_team2", "Seed_team1", "Seed_team2", "pred", "proba"]].iloc[i]
        mirror = data[(data["TeamName"] == game["TeamName_team2"]) & (data["TeamName_team2"] == game["TeamName"])]
        prob1 = float((game["proba"] + (1 - mirror["proba"])) / 2)
        prob2 = float(((1-game["proba"]) + mirror["proba"]) / 2)
        pred_array.append([game["TeamName"], game["Seed_team1"], game["TeamName_team2"], game["Seed_team2"], prob1, prob2, int(break_even_moneyline(prob1)), int(break_even_moneyline(prob2))])
    
    return pd.DataFrame(pred_array, columns = predictions.columns)

In [101]:
pred_df = predict_probs_and_moneylines(test_with_names)

pred_df.to_csv("predictions/predictions.csv")

In [102]:
pred_df = pd.read_csv("predictions_2.csv")

In [106]:
round_1_matchups = [
    ("Gonzaga", "Georgia St"),
    ("Boise St", "Memphis"),
    ("Connecticut", "New Mexico St"),
    ("Arkansas", "Vermont"),
    ("Alabama", "Notre Dame"),
    ("Texas Tech", "Montana St"),
    ("Michigan St", "Davidson"),
    ("Duke", "CS Fullerton"),
    ("Baylor", "Norfolk St"),
    ("North Carolina", "Marquette"),
    ("St Mary's CA", "Indiana"),
    ("UCLA", "Akron"),
    ("Texas", "Virginia Tech"),
    ("Purdue", "Yale"),
    ("Murray St", "San Francisco"),
    ("Kentucky", "St Peter's"),
    ("Arizona", "Wright St"),
    ("Seton Hall", "TCU"),
    ("Houston", "UAB"),
    ("Illinois", "Chattanooga"),
    ("Colorado St", "Michigan"),
    ("Tennessee", "Longwood"),
    ("Ohio St", "Loyola-Chicago"),
    ("Villanova", "Delaware"),
    ("Kansas", "TX Southern"),
    ("San Diego St", "Creighton"),
    ("Iowa", "Richmond"),
    ("Providence", "S Dakota St"),
    ("LSU", "Iowa St"),
    ("Wisconsin", "Colgate"),
    ("USC", "Miami FL"),
    ("Auburn", "Jacksonville St")
]
final_four = [
    ("Kansas", "Auburn"),
    ("Kentucky", "Baylor"),
    ("Auburn", "Arizona"),
    ("Tennessee", "Villanova")
]

In [107]:
def pretty_print_matchups(matchups, include_moneyline=True):
    for (x, y) in matchups:
        game = pred_df[(pred_df["Team1"] == x) & (pred_df["Team2"]==y)].iloc[0]
        print(game["Team1"] + " (" + str(game["Seed1"]) + ") vs. " + game["Team2"] + " (" + str(game["Seed2"]) + "):")
        print("{:.2%}".format(game["Win%1"]) + " : {:.2%}".format(game["Win%2"]))
        if (include_moneyline):
            print("Moneyline: " + str(abs(game["ML1"])))
        print()

In [109]:
pretty_print_matchups(round_1_matchups)

Gonzaga (1) vs. Georgia St (16):
96.66% : 3.34%
Moneyline: 2890

Boise St (8) vs. Memphis (9):
46.55% : 53.45%
Moneyline: 114

Connecticut (5) vs. New Mexico St (12):
77.46% : 22.54%
Moneyline: 343

Arkansas (4) vs. Vermont (13):
77.57% : 22.43%
Moneyline: 345

Alabama (6) vs. Notre Dame (11):
57.79% : 42.21%
Moneyline: 136

Texas Tech (3) vs. Montana St (14):
86.99% : 13.01%
Moneyline: 668

Michigan St (7) vs. Davidson (10):
53.90% : 46.10%
Moneyline: 116

Duke (2) vs. CS Fullerton (15):
92.58% : 7.42%
Moneyline: 1247

Baylor (1) vs. Norfolk St (16):
96.12% : 3.88%
Moneyline: 2479

North Carolina (8) vs. Marquette (9):
55.64% : 44.36%
Moneyline: 125

St Mary's CA (5) vs. Indiana (12):
60.82% : 39.18%
Moneyline: 155

UCLA (4) vs. Akron (13):
88.56% : 11.44%
Moneyline: 774

Texas (6) vs. Virginia Tech (11):
62.43% : 37.57%
Moneyline: 166

Purdue (3) vs. Yale (14):
91.88% : 8.12%
Moneyline: 1130

Murray St (7) vs. San Francisco (10):
54.68% : 45.32%
Moneyline: 120

Kentucky (2) vs. St Pe